In [15]:
!pip install tweet-preprocessor
!pip install yfinance

     |████████████████████████████████| 5.5MB 9.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=f7c125569d043323c203eaaa366272c2c0f8f55244cdad2c8f63a88c9675ed26
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [16]:
import pandas as pd
import matplotlib.pyplot as polt
from textblob import TextBlob
from datetime import datetime, timedelta
import requests
import pandas as pd
import preprocessor as p
import yfinance as yf

In [5]:
ticker = 'GME'
company = 'Gamestop'

In [ ]:
# read bearer token for authentication
with open('bearer.txt') as fp: #The bearer.txt file is on the github
    BEARER_TOKEN = fp.read()
# setup the API request
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
headers = {'authorization': f'Bearer {BEARER_TOKEN}'}
params = {
    'query': '('+ ticker+') (lang:en)',
    'max_results': '100',
    'tweet.fields': 'created_at,lang'
}

dtformat = '%Y-%m-%dT%H:%M:%SZ'  # the date format string required by twitter

# we use this function to subtract 60 mins from our datetime string
def time_travel(now, mins):
    now = datetime.strptime(now, dtformat)
    back_in_time = now - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)

def get_data(tweet):
    data = {
        'id': tweet['id'],
        'created_at': tweet['created_at'],
        'text': tweet['text']
    }
    return data
    
now = datetime.now() - timedelta(seconds=20) # get the current datetime
last_week = now - timedelta(days=1)  # datetime one day ago = the finish line
now = now.strftime(dtformat)  # convert now datetime to format for API

df = pd.DataFrame()  # initialize dataframe to store tweets
while True:
    if datetime.strptime(now, dtformat) < last_week:
        break
    pre60 = time_travel(now, 60)  # get 60 minutes before 'now'
    # assign from and to datetime parameters for the API
    params['start_time'] = pre60
    params['end_time'] = now
    response = requests.get(endpoint,
                            params=params,
                            headers=headers)  # send the request
    now = pre60  # move the window 60 minutes earlier
    # iteratively append our tweet data to our dataframe
    for tweet in response.json()['data']:
        row = get_data(tweet)
        df = df.append(row, ignore_index=True)

In [6]:
# Save csv file here
filename = company + ".csv"
df.to_csv(filename,index = False)

NameError: ignored

In [3]:
df.head()

NameError: ignored

In [7]:
# Load csv file here
filename = company + ".csv"
df = pd.read_csv(filename)

In [8]:
sentiment = []
scores = []
for tweet in df.text:
    # tweet = p.clean(tweet) #removes emojis etc
    analysis = TextBlob(tweet)
    score = analysis.sentiment.polarity 
    scores.append(score)
    if score > 0:
        sentiment.append("positive")
    elif score == 0:
        sentiment.append("neutral")
    else:
        sentiment.append("negative")
df["sentiment"] = sentiment
df["score"] = scores

In [9]:
pos = 100 * len(df[df['sentiment'].str.contains('positive')]) / len(df)
neg = 100 * len(df[df['sentiment'].str.contains('negative')]) / len(df)
neu = 100 * len(df[df['sentiment'].str.contains('neutral')]) / len(df)
print("Positive tweet {} %".format(pos))
print("Negative tweets {} %".format(neg))
print("Neutral tweets {} % ".format(neu))

Positive tweet 41.40950792326939 %
Negative tweets 13.261050875729774 %
Neutral tweets 45.32944120100083 % 


In [12]:
df = df.sort_values(by=['score'])

In [13]:
dis = df.filter(["text","score"], axis=1)

In [14]:
dis.iloc[:5]

,text,score
145,The media will do one of two things regarding ...,-1.0000
1854,@StockMktMedia @markets You know who DIDN'T sh...,-0.9375
723,RT @madaznfootballr: Trader 'madazmoney' trade...,-0.8000
713,RT @madaznfootballr: Trader 'madazmoney' trade...,-0.8000
1776,RT @madaznfootballr: Trader 'madazmoney' trade...,-0.8000


Not sure how to work with yahoo finance yet. Below does not work.

In [ ]:

gme = yf.Ticker("GME")
gme_stock = gme.history(
    start=(data['created_at'].min()).strftime('%Y-%m-%d'),
    end=data['created_at'].max().strftime('%Y-%m-%d'),
    interval='60m'
).reset_index()